[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ClimateMatchAcademy/course-content/blob/main/tutorials/W2D5_ClimateResponse-AdaptationImpact/Get_Crop_Harvest_Data.ipynb)

In [ ]:
# install cropharvest from PyPI
!pip install cropharvest -q

In [ ]:
# !pip freeze | grep cropharvest

In [ ]:
# there is a pull request pending that means we wouldn't need this line soon
import collections.abc

collections.Hashable = collections.abc.Hashable

In [ ]:
# imports
from cropharvest.datasets import CropHarvest
from cropharvest.inference import Inference
from pathlib import Path

import xarray as xr
import numpy as np

import requests
import tempfile

mkdir: cannot create directory ‘data’: File exists


In [ ]:
# get the training dataset (there are three returned for different regions, and togo is the last one)
# this can take a while to download
_, _, togo_training_dataset = CropHarvest.create_benchmark_datasets(DATA_DIR)
togo_training_dataset

CropHarvestEval(Togo_crop, togo-eval)

In [ ]:
# break apart the returned tuple into regressors and values
X_train, y_train = togo_training_dataset.as_array()

# look at the shape of the data
X_train.shape, y_train.shape

((1290, 12, 18), (1290,))

In [ ]:
# The data is arranged as locations, months, bands/climate variables
# Just use data from the first month:
X_train = X_train[:, 0, :]
X_train.shape

(1290, 18)

In [ ]:
# See what the 18 bands/claimte variables are:
from cropharvest.bands import BANDS

BANDS

['VV',
 'VH',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'temperature_2m',
 'total_precipitation',
 'elevation',
 'slope',
 'NDVI']

In [ ]:
# We just want to use the standard Sentinel bands plus NDVI:
X_train = X_train[:, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17]]
reduced_BANDS = BANDS[2:13] + [BANDS[17]]
reduced_BANDS

['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'NDVI']

In [ ]:
# make training data set
training_data_ds = xr.Dataset()
training_data_ds["X_train"] = xr.DataArray(
    X_train, dims=["loc", "bands"], coords=[np.arange(len(X_train)), reduced_BANDS]
)
training_data_ds["y_train"] = xr.DataArray(
    y_train, dims=["loc"], coords=[np.arange(len(y_train))]
)
training_data_ds

<xarray.Dataset>
Dimensions:  (loc: 1290, bands: 12)
Coordinates:
  * loc      (loc) int64 0 1 2 3 4 5 6 7 ... 1283 1284 1285 1286 1287 1288 1289
  * bands    (bands) <U4 'B2' 'B3' 'B4' 'B5' 'B6' ... 'B9' 'B11' 'B12' 'NDVI'
Data variables:
    X_train  (loc, bands) float64 -0.06259 -0.106 -0.06556 ... -0.2384 0.33
    y_train  (loc) int64 1 1 1 1 1 1 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
# save to netcdf file
training_data_ds.to_netcdf("training.nc")

In [ ]:
# get the test data
# the for loop is necessary because .test_data is a 'generator'
for _, test_instance in togo_training_dataset.test_data():
    X_test = test_instance.x
    y_test = test_instance.y

# look at the shape of the data
X_test.shape, y_test.shape

((306, 12, 18), (306,))

In [ ]:
# Use just the first month and the designated bands:
X_test = X_test[:, 0, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17]]

In [ ]:
# Make test dataset
test_data_ds = xr.Dataset()
test_data_ds["X_test"] = xr.DataArray(
    X_test, dims=["loc", "bands"], coords=[np.arange(len(X_test)), reduced_BANDS]
)
test_data_ds["y_test"] = xr.DataArray(
    y_test, dims=["loc"], coords=[np.arange(len(y_test))]
)
test_data_ds

<xarray.Dataset>
Dimensions:  (loc: 306, bands: 12)
Coordinates:
  * loc      (loc) int64 0 1 2 3 4 5 6 7 8 ... 298 299 300 301 302 303 304 305
  * bands    (bands) <U4 'B2' 'B3' 'B4' 'B5' 'B6' ... 'B9' 'B11' 'B12' 'NDVI'
Data variables:
    X_test   (loc, bands) float64 -0.2125 -0.2114 -0.1286 ... 0.2059 -0.5421
    y_test   (loc) int64 0 0 0 0 0 1 1 0 1 0 0 0 1 ... 0 1 1 0 0 1 1 0 0 1 0 0 1

In [ ]:
# save to netcdf file
test_data_ds.to_netcdf("test.nc")